# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 29 2022 10:19AM,253994,10,MSP217079,"Methapharm, Inc.",Released
1,Dec 29 2022 10:19AM,253994,10,MSP217047,"Methapharm, Inc.",Released
2,Dec 29 2022 10:19AM,253994,10,MSP217065,"Methapharm, Inc.",Released
3,Dec 29 2022 10:19AM,253994,10,MSP217086,"Methapharm, Inc.",Released
4,Dec 29 2022 10:19AM,253994,10,MSP217091,"Methapharm, Inc.",Released
5,Dec 29 2022 10:19AM,253994,10,MSP217093,"Methapharm, Inc.",Released
6,Dec 29 2022 10:15AM,253993,17,8960801,"Aceragen, Inc.",Released
7,Dec 29 2022 10:11AM,253992,17,8963979,"Breckenridge Pharmaceutical, Inc.",Released
8,Dec 29 2022 9:58AM,253989,20,ATR-I000038,"HVL, LLC dba Atrium Innovations",Released
9,Dec 29 2022 9:51AM,253988,19,TR-RA-01032023,"GCH Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,253988,Released,1
36,253989,Released,1
37,253992,Released,1
38,253993,Released,1
39,253994,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253988,NaN,1.0
253989,NaN,1.0
253992,NaN,1.0
253993,NaN,1.0
253994,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253903,0.0,1.0
253904,0.0,7.0
253905,0.0,11.0
253908,0.0,1.0
253909,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253903,0,1
253904,0,7
253905,0,11
253908,0,1
253909,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253903,0,1
1,253904,0,7
2,253905,0,11
3,253908,0,1
4,253909,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253903,,1
1,253904,,7
2,253905,,11
3,253908,,1
4,253909,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 29 2022 10:19AM,253994,10,"Methapharm, Inc."
6,Dec 29 2022 10:15AM,253993,17,"Aceragen, Inc."
7,Dec 29 2022 10:11AM,253992,17,"Breckenridge Pharmaceutical, Inc."
8,Dec 29 2022 9:58AM,253989,20,"HVL, LLC dba Atrium Innovations"
9,Dec 29 2022 9:51AM,253988,19,"GCH Granules USA, Inc."
10,Dec 29 2022 9:31AM,253985,16,"Emersa Waterbox, LLC"
11,Dec 29 2022 9:25AM,253984,10,"Citieffe, Inc."
12,Dec 29 2022 9:01AM,253983,10,Emerginnova
14,Dec 29 2022 9:01AM,253982,10,ISDIN Corporation
17,Dec 29 2022 8:59AM,253981,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 29 2022 10:19AM,253994,10,"Methapharm, Inc.",,6
1,Dec 29 2022 10:15AM,253993,17,"Aceragen, Inc.",,1
2,Dec 29 2022 10:11AM,253992,17,"Breckenridge Pharmaceutical, Inc.",,1
3,Dec 29 2022 9:58AM,253989,20,"HVL, LLC dba Atrium Innovations",,1
4,Dec 29 2022 9:51AM,253988,19,"GCH Granules USA, Inc.",,1
5,Dec 29 2022 9:31AM,253985,16,"Emersa Waterbox, LLC",,1
6,Dec 29 2022 9:25AM,253984,10,"Citieffe, Inc.",,1
7,Dec 29 2022 9:01AM,253983,10,Emerginnova,,2
8,Dec 29 2022 9:01AM,253982,10,ISDIN Corporation,,3
9,Dec 29 2022 8:59AM,253981,10,ISDIN Corporation,,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 10:19AM,253994,10,"Methapharm, Inc.",6,
1,Dec 29 2022 10:15AM,253993,17,"Aceragen, Inc.",1,
2,Dec 29 2022 10:11AM,253992,17,"Breckenridge Pharmaceutical, Inc.",1,
3,Dec 29 2022 9:58AM,253989,20,"HVL, LLC dba Atrium Innovations",1,
4,Dec 29 2022 9:51AM,253988,19,"GCH Granules USA, Inc.",1,
5,Dec 29 2022 9:31AM,253985,16,"Emersa Waterbox, LLC",1,
6,Dec 29 2022 9:25AM,253984,10,"Citieffe, Inc.",1,
7,Dec 29 2022 9:01AM,253983,10,Emerginnova,2,
8,Dec 29 2022 9:01AM,253982,10,ISDIN Corporation,3,
9,Dec 29 2022 8:59AM,253981,10,ISDIN Corporation,13,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 10:19AM,253994,10,"Methapharm, Inc.",6,
1,Dec 29 2022 10:15AM,253993,17,"Aceragen, Inc.",1,
2,Dec 29 2022 10:11AM,253992,17,"Breckenridge Pharmaceutical, Inc.",1,
3,Dec 29 2022 9:58AM,253989,20,"HVL, LLC dba Atrium Innovations",1,
4,Dec 29 2022 9:51AM,253988,19,"GCH Granules USA, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 10:19AM,253994,10,"Methapharm, Inc.",6.0,NaN
1,Dec 29 2022 10:15AM,253993,17,"Aceragen, Inc.",1.0,NaN
2,Dec 29 2022 10:11AM,253992,17,"Breckenridge Pharmaceutical, Inc.",1.0,NaN
3,Dec 29 2022 9:58AM,253989,20,"HVL, LLC dba Atrium Innovations",1.0,NaN
4,Dec 29 2022 9:51AM,253988,19,"GCH Granules USA, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 10:19AM,253994,10,"Methapharm, Inc.",6.0,0.0
1,Dec 29 2022 10:15AM,253993,17,"Aceragen, Inc.",1.0,0.0
2,Dec 29 2022 10:11AM,253992,17,"Breckenridge Pharmaceutical, Inc.",1.0,0.0
3,Dec 29 2022 9:58AM,253989,20,"HVL, LLC dba Atrium Innovations",1.0,0.0
4,Dec 29 2022 9:51AM,253988,19,"GCH Granules USA, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,7364317,161.0,0.0
16,253985,1.0,0.0
17,1269867,5.0,1.0
19,761903,2.0,1.0
20,507963,4.0,0.0
21,253916,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,7364317,161.0,0.0
1,16,253985,1.0,0.0
2,17,1269867,5.0,1.0
3,19,761903,2.0,1.0
4,20,507963,4.0,0.0
5,21,253916,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,161.0,0.0
1,16,1.0,0.0
2,17,5.0,1.0
3,19,2.0,1.0
4,20,4.0,0.0
5,21,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,161.0
1,16,Released,1.0
2,17,Released,5.0
3,19,Released,2.0
4,20,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,17,19,20,21
Status,,,,,,
Executing,0.0,0.0,1.0,1.0,0.0,1.0
Released,161.0,1.0,5.0,2.0,4.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,17,19,20,21
0,Executing,0.0,0.0,1.0,1.0,0.0,1.0
1,Released,161.0,1.0,5.0,2.0,4.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,17,19,20,21
0,Executing,0.0,0.0,1.0,1.0,0.0,1.0
1,Released,161.0,1.0,5.0,2.0,4.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()